In [1]:
import numpy as np
import torch
import torchvision

In [2]:
# Prameters
input_size = 28
num_classes = 10
num_epoch = 5
batch_size = 64

In [19]:
# Common parameter
kernel_size = 5
stride = 1
padding = 2
max_pool_size = 2
learning_rate = 0.001

# Convolutional Layer 1

layer1_in_channels = 1 # Gray picture
layer1_out_channels = 10 # NUM of Filter

# Convolutional Layer 2

layer2_in_channels = layer1_out_channels
layer2_out_channels = 32

# Output size
layer_1_out_size = ((28 - kernel_size + 2 * padding) / stride) + 1
layer_1_out_size = layer_1_out_size / max_pool_size 
layer_2_out_size = ((layer_1_out_size - kernel_size + 2 * padding) / stride) + 1
layer_2_out_size = layer_2_out_size / max_pool_size 


In [4]:
# Load data

train_data = torchvision.datasets.MNIST(root = './data', train = True, 
                                        transform = torchvision.transforms.ToTensor())
test_data = torchvision.datasets.MNIST(root = './data', train = False,
                                      transform = torchvision.transforms.ToTensor())

# Data Loader

train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = False)

In [36]:
# Create model

class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = torch.nn.Sequential(torch.nn.Conv2d(in_channels = layer1_in_channels, 
                                                   out_channels = layer1_out_channels,
                                                   kernel_size = kernel_size,
                                                   stride = stride,
                                                   padding = padding,), 
                                        torch.nn.ReLU(),
                                        torch.nn.MaxPool2d(kernel_size = max_pool_size))
        
        self.conv2 = torch.nn.Sequential(torch.nn.Conv2d(in_channels = layer1_out_channels, 
                                                        out_channels = layer2_out_channels,
                                                        kernel_size = kernel_size,
                                                        stride = stride,
                                                        padding = padding,),
                                        torch.nn.ReLU(),
                                        torch.nn.MaxPool2d(kernel_size = max_pool_size),)
        self.out = torch.nn.Linear(int(layer2_out_channels * layer_2_out_size * layer_2_out_size), 10)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1) # x.size = (batch_size, 32*7*7)
        out = self.out(x)
        return out
        

In [42]:
def accuracy(predictions, labels):
    # torch.max()[0] = return max value; [1] means return index of max value
    pred = torch.max(predictions.data, dim = 1)[1]
    # Check if predicitons and labels match
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [50]:
net = CNN()

In [51]:
# Loss function
criterion = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(params = net.parameters(), lr = learning_rate)

In [52]:
for epoch in range(num_epoch):
    train_rights = []
    
    # x = data; label = target
    # Enumerate will automically generate index like 0,1,2, ...
    for batch_idx, (data, target) in enumerate(train_loader):
        net.train()
        output = net(data)
        loss = criterion(output, target)
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        
        right = accuracy(output, target)
        train_rights.append(right)
        
        if batch_idx % 100 == 0:
            
            net.eval()
            val_rights = []
            
            for(data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)
                
            # Calculate accuracy
            # tup[0] = rights, tup[1] = length of labels
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights] ))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))
            
            print('Epoch: {}, Loss: {}, Train_acc: {:.6f}, Val_acc: {:.6f}'.format(epoch, loss.item(), 
                                                                           train_r[0].numpy()/train_r[1],
                                                                          val_r[0].numpy()/val_r[1]) )

Epoch: 0, Loss: 2.2963602542877197, Train_acc: 0.078125, Val_acc: 0.107400
Epoch: 0, Loss: 0.40282484889030457, Train_acc: 0.748917, Val_acc: 0.894400
Epoch: 0, Loss: 0.18360526859760284, Train_acc: 0.834188, Val_acc: 0.949700
Epoch: 0, Loss: 0.34068697690963745, Train_acc: 0.872612, Val_acc: 0.963000
Epoch: 0, Loss: 0.07349003851413727, Train_acc: 0.895729, Val_acc: 0.968600
Epoch: 0, Loss: 0.09858673065900803, Train_acc: 0.909712, Val_acc: 0.968200
Epoch: 0, Loss: 0.16299180686473846, Train_acc: 0.920055, Val_acc: 0.972700
Epoch: 0, Loss: 0.08103194087743759, Train_acc: 0.927314, Val_acc: 0.975900
Epoch: 0, Loss: 0.04780641198158264, Train_acc: 0.933306, Val_acc: 0.977100
Epoch: 0, Loss: 0.058251019567251205, Train_acc: 0.937743, Val_acc: 0.981500
Epoch: 1, Loss: 0.021709691733121872, Train_acc: 0.984375, Val_acc: 0.978700
Epoch: 1, Loss: 0.08788266032934189, Train_acc: 0.981590, Val_acc: 0.981600
Epoch: 1, Loss: 0.03779156506061554, Train_acc: 0.981032, Val_acc: 0.979200


KeyboardInterrupt: 